In [ ]:
!pip install openai==0.28
!pip install -U datasets
!pip install datasets --quiet
!pip install openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.93.0
    Uninstalling openai-1.93.0:
      Successfully uninstalled openai-1.93.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+c

In [ ]:
import pandas as pd
import openai
import time
import pandas
import re
import os


In [ ]:

from google.colab import files
uploaded = files.upload()


Saving author_334_posts.csv to author_334_posts.csv


In [ ]:

# Set your Groq API key
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = "https://api.groq.com/openai/v1"

# Send a prompt to LLaMA 3
response = openai.ChatCompletion.create(
    model="llama3-70b-8192",  # or "mixtral-8x7b-32768"
    messages=[{"role": "user", "content": "give the score for score for o c e a n personalty trait of sentence, hey well done you are really good in this."}],
    temperature=0.7
)


In [ ]:
import pandas as pd
import openai
import time
import re

# === Load data from row 1000 to 1099 ===
df = pd.read_csv("author_334_posts.csv").iloc[1000:1100].copy()
if 'input_text' not in df.columns:
    raise ValueError("❌ 'input_text' column not found in the CSV file.")

# === Define strict prompt ===
def make_prompt(text):
    return (
        "You are a behavioral finance expert.\n\n"
        "Analyze the following social media post to determine if it reflects **Loss Aversion** bias.\n\n"
        "Loss Aversion is only present when a user clearly describes a personal investment decision (such as buying, holding, or selling a financial asset), "
        "and shows emotional discomfort related to a potential or realized financial loss.\n\n"
        "Do NOT classify as Loss Aversion if the user is only discussing general market trends, news, or other people’s actions without describing their own decisions.\n\n"
        "Only respond if both criteria are clearly met.\n\n"
        "Strictly follow this output format:\n\n"
        "Detected_Biases: <'Loss Aversion' or 'None'>\n\n"
        "Reasoning Paragraph:\n"
        "<One-paragraph explanation. If bias is 'None', explain why.>\n\n"
        "Statement:\n"
        f"{text}"
    )

# === Parse model output correctly ===
def parse_response(text):
    try:
        bias = re.search(r"Detected_Biases:\s*(.+?)\n", text).group(1).strip()
        reasoning = re.search(r"Reasoning Paragraph:\s*(.+?)\n\s*Statement:", text, re.DOTALL).group(1).strip()
    except:
        bias = "None"
        reasoning = ""
    return bias, reasoning

# === Initialize storage ===
biases, reasons = [], []

# === API call limits ===
call_count = 0
calls_per_minute = 30
start_time = time.time()

# === Loop through rows ===
for i, row in df.iterrows():
    if call_count % calls_per_minute == 0 and call_count > 0:
        elapsed = time.time() - start_time
        if elapsed < 60:
            time.sleep(60 - elapsed)
        start_time = time.time()

    prompt = make_prompt(row["input_text"])

    retries = 0
    while True:
        try:
            response = openai.ChatCompletion.create(
                model="meta-llama/llama-4-scout-17b-16e-instruct",
                messages=[
                    {"role": "system", "content": "You are a behavioral finance expert trained to detect financial decision-making biases."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.1
            )
            call_count += 1
            content = response["choices"][0]["message"]["content"].strip()

            # DEBUG: Print prompt and model output
            print(f"\n📥 Prompt (row {i}):\n{prompt}")
            print(f"\n📤 Response (row {i}):\n{content}\n{'-'*80}")

            bias, reasoning = parse_response(content)

            biases.append(bias)
            reasons.append(reasoning)
            break
        except Exception as e:
            retries += 1
            if retries > 3:
                print(f"❌ Failed at row {i}: {e}")
                biases.append("None")
                reasons.append("")
                break
            time.sleep(2 ** retries)

    time.sleep(2)

# === Save only those rows where bias is 'Loss Aversion' and reasoning is non-empty ===
df["detected_biases"] = biases
df["reasoning_paragraph"] = reasons

df_filtered = df[
    (df["detected_biases"] == "Loss Aversion") &
    (df["reasoning_paragraph"].str.strip() != "")
]

df_filtered.to_csv("strict_loss_aversion_output_1000_1099.csv", index=False)
print("✅ Saved filtered results to strict_loss_aversion_output_1000_1099.csv")



📥 Prompt (row 1000):
You are a behavioral finance expert.

Analyze the following social media post to determine if it reflects **Loss Aversion** bias.

Loss Aversion is only present when a user clearly describes a personal investment decision (such as buying, holding, or selling a financial asset), and shows emotional discomfort related to a potential or realized financial loss.

⚠️ Do NOT classify as Loss Aversion if the user is only discussing general market trends, news, or other people’s actions without describing their own decisions.

Only respond if both criteria are clearly met.

Strictly follow this output format:

Detected_Biases: <'Loss Aversion' or 'None'>

Reasoning Paragraph:
<One-paragraph explanation. If bias is 'None', explain why.>

Statement:
With 10 Year Treasury Bill at a RECORD LOW 1% dividends in Bond Funds will drop to near zero

Most of us old folks who are retired have 40-60% of our portfolio in Bond Mutual Funds and ETF's.  Typically much of their return is t

In [ ]:
import pandas as pd
import openai
import time
import re

# === Loss Aversion behavioral indicators ===
loss_aversion_signals = {
    "Decision Context": "The user explicitly discusses a personal financial decision (buying, holding, or selling an asset).",
    "Emotional Language Around Loss": "Mentions regret, fear, pain, can't afford, devastated, etc.",
    "Comparative Framing": "Mentions what could have been gained vs. what was lost.",
    "Risk-Averse Behavior": "Avoids selling at a loss or hesitates reinvesting after losses.",
    "Endowment Effect Indicators": "Overvalues their own assets; reluctant to sell due to ownership.",
    "Status Quo Bias": "Prefers to do nothing to avoid further losses.",
    "Temporal References": "Cites past losses as reasons for current caution or inaction."
}

loss_aversion_keywords = {
    "Fear of loss": "Avoiding actions that could result in financial loss, even if potential gains outweigh the risks.",
    "Risk aversion": "Preferring smaller, certain returns over larger but uncertain ones due to fear of loss.",
    "Holding losers": "Not selling underperforming assets to avoid the pain of realizing a loss.",
    "Selling winners too early": "Quickly locking in profits to avoid potential future loss.",
    "Disproportionate pain from losses": "Losses feel psychologically stronger than equivalent gains.",
    "Avoiding risky but rational trades": "Skipping profitable investments due to emotional sensitivity to possible losses."
}

# === Prompt Generator ===
def make_prompt(text):
    signal_text = "\n".join([f"- {k}: {v}" for k, v in loss_aversion_signals.items()])
    keyword_text = "\n".join([f"- {k}: {v}" for k, v in loss_aversion_keywords.items()])

    return (
        "You are a behavioral finance expert.\n\n"
        "Your task is to analyze the following social media post for the presence of **Loss Aversion** bias.\n\n"
        "Loss Aversion only applies if the user describes their own financial decision (buying/selling/holding assets) AND expresses emotional discomfort or risk-avoidant behavior around potential or realized losses.\n\n"
        "❌ Do NOT return 'Loss Aversion' if there's no decision context.\n\n"
        "Look for the following signs:\n"
        f"{signal_text}\n\n"
        "You can also use these Loss Aversion keywords:\n"
        f"{keyword_text}\n\n"
        "Respond strictly in this format:\n\n"
        "Detected_Biases: <'Loss Aversion' or 'None'>\n\n"
        "Reasoning Paragraph:\n"
        "<One paragraph explaining your decision. If 'None', justify why.>\n\n"
        "Statement:\n"
        f"{text}"
    )

# === Response Parser ===
def parse_response(text):
    try:
        bias = re.search(r"Detected_Biases:\s*(.+?)\n", text).group(1).strip()
        reasoning = re.search(r"Reasoning Paragraph:\s*(.+?)\n\s*Statement:", text, re.DOTALL).group(1).strip()
    except:
        bias = "None"
        reasoning = ""
    return bias, reasoning

# === Load Dataset ===
df = pd.read_csv("author_334_posts.csv").iloc[1000:1002].copy()
if 'input_text' not in df.columns:
    raise ValueError("❌ 'input_text' column not found.")

results, reasons = [], []
call_count = 0
calls_per_minute = 30
start_time = time.time()

# === Main Loop ===
for i, row in df.iterrows():
    if call_count % calls_per_minute == 0 and call_count > 0:
        elapsed = time.time() - start_time
        if elapsed < 60:
            time.sleep(60 - elapsed)
        start_time = time.time()

    prompt = make_prompt(row['input_text'])
    retries = 0
    while True:
        try:
            response = openai.ChatCompletion.create(
                model="meta-llama/llama-4-scout-17b-16e-instruct",
                messages=[
                    {"role": "system", "content": "You are a behavioral finance expert trained to detect financial decision-making biases."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.1
            )
            call_count += 1

            content = response['choices'][0]['message']['content'].strip()

            # Print results for verification
            # print(f"\n📥 Prompt (row {i}):\n{prompt}")
            print(f"\n📤 Response (row {i}):\n{content}\n{'='*80}")

            bias, reasoning = parse_response(content)
            results.append(bias)
            reasons.append(reasoning)
            break
        except Exception as e:
            retries += 1
            if retries > 3:
                print(f"❌ Failed at row {i}: {e}")
                results.append("None")
                reasons.append("")
                break
            time.sleep(2 ** retries)

    time.sleep(2)


📥 Prompt (row 1000):
You are a behavioral finance expert.

Your task is to analyze the following social media post for the presence of **Loss Aversion** bias.

Loss Aversion only applies if the user describes their own financial decision (buying/selling/holding assets) AND expresses emotional discomfort or risk-avoidant behavior around potential or realized losses.

❌ Do NOT return 'Loss Aversion' if there's no decision context.

Look for the following signs:
- Decision Context: The user explicitly discusses a personal financial decision (buying, holding, or selling an asset).
- Emotional Language Around Loss: Mentions regret, fear, pain, can't afford, devastated, etc.
- Comparative Framing: Mentions what could have been gained vs. what was lost.
- Risk-Averse Behavior: Avoids selling at a loss or hesitates reinvesting after losses.
- Endowment Effect Indicators: Overvalues their own assets; reluctant to sell due to ownership.
- Status Quo Bias: Prefers to do nothing to avoid further 

In [ ]:


# === Save Results with Only Reasoned Rows ===
df["detected_biases"] = results
df["reasoning_paragraph"] = reasons

df_filtered = df[
    df["reasoning_paragraph"].str.strip() != ""
]


df_filtered.to_csv("strict_loss_aversion_with_signals_2.csv", index=False)
print("✅ Saved results to strict_loss_aversion_with_signals_2.csv")


✅ Saved results to strict_loss_aversion_with_signals_2.csv


In [ ]:

files.download('strict_loss_aversion_with_signals_2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# def make_prompt(text):
#     return (
#         "You are a behavioral finance expert. Your job is to analyze a given statement and determine whether each of the following financial biases is present or not. "
#         "The biases are:\n"
#         "1. Loss Aversion\n2. Confirmation Bias\n3. Overconfidence\n4. Herd Mentality\n5. Anchoring\n"
#         "6. Mental Accounting\n7. Status Quo Bias\n8. Recency Bias\n9. Availability Bias\n10. Endowment Effect\n\n"
#         "For each bias, output 1 if it is present and 0 if not. After that, provide a one-paragraph reasoning for your choices.\n"
#         "Finally, write a single-sentence summary of the user's overall biasing behavior.\n"
#         "Strictly follow this format:\n\n"
#         "Loss_Aversion: <0 or 1>\n"
#         "Confirmation_Bias: <0 or 1>\n"
#         "Overconfidence: <0 or 1>\n"
#         "Herd_Mentality: <0 or 1>\n"
#         "Anchoring: <0 or 1>\n"
#         "Mental_Accounting: <0 or 1>\n"
#         "Status_Quo: <0 or 1>\n"
#         "Recency_Bias: <0 or 1>\n"
#         "Availability_Bias: <0 or 1>\n"
#         "Endowment_Effect: <0 or 1>\n\n"
#         "Reasoning Paragraph:\n"
#         "<One-paragraph explanation>\n\n"
#         "Bias Summary:\n"
#         "<One sentence summarizing the main observed biases>\n\n"
#         "Statement:\n"
#         f"{text}"
#     )

In [ ]:
def make_prompt(text):
    return (
        "You are a behavioral finance expert. Your job is to analyze a given statement and determine whether each of the following financial biases is present or not. "

        "Statement:\n"
        f"{text}"
    )

In [ ]:
def make_prompt(text):
    return (
        "You are a behavioral finance expert. Analyze the following statement and determine if any of the financial decision-making biases from the list below are present.\n\n"
        "Only consider the following 16 biases (do not include any other bias names):\n"
        f"{', '.join(bias_list)}.\n\n"
        "If you detect biases, list only the exact names from the list above. "
        "Do not include any bias not in the list. Do not invent or paraphrase bias names.\n\n"
        "Then write one paragraph explaining your reasoning and a one-sentence summary of the user's overall biasing behavior.\n\n"
        "Strictly follow this format:\n\n"
        "Detected_Biases: <comma-separated exact bias names from the list above, or 'None'>\n\n"
        "Reasoning Paragraph:\n"
        "<One-paragraph explanation>\n\n"
        "Bias Summary:\n"
        "<One sentence summarizing the observed biasing behavior>\n\n"
        "Statement:\n"
        f"{text}"
    )


In [ ]:
import pandas as pd
import openai
import time
import re

# === Keywords for Loss Aversion Bias ===
status_quo_keywords = {
    "Preference for current state": "A tendency to favor existing situations over making changes.",
    "Inertia in decision-making": "Delaying or avoiding decisions due to effort or discomfort with change.",
    "Resistance to change": "Rejecting new options or strategies even when they may be better.",
    "Default choice preference": "Choosing default or pre-set options rather than actively evaluating alternatives.",
    "Change avoidance": "Avoiding changes in investment or plans due to perceived risk or discomfort.",
    "Comfort in familiarity": "Sticking with known investments or strategies because they feel safe.",
    "Regret minimization": "Avoiding decisions that could lead to regret, even if change might be beneficial."
}


# === Prompt generator with keywords embedded ===
def make_prompt(text):
    keyword_text = "\n".join([f"- {k}: {v}" for k, v in status_quo_keywords.items()])
    return (
        "You are a behavioral finance expert. Your task is to analyze the statement below and determine whether the investor shows signs of **Status Quo Bias**.\n\n"
        "Status Quo Bias refers to the irrational preference for the current state of affairs and resistance to change, even when change may be beneficial.\n\n"
        "Use the following behavioral indicators (keywords) to guide your judgment:\n"
        f"{keyword_text}\n\n"
        "If Status Quo Bias is present, explain why in a paragraph and summarize the behavior in one sentence.\n\n"
        "Strictly follow this format:\n\n"
        "Detected_Biases: <'Status Quo Bias' or 'None'>\n\n"
        "Reasoning Paragraph:\n"
        "<One-paragraph explanation>\n\n"
        "Bias Summary:\n"
        "<One sentence summarizing the observed behavior>\n\n"
        "Statement:\n"
        f"{text}"
    )

# === Response parser ===
def parse_response(text):
    bias_match = re.search(r"Detected_Biases:\s*(.+?)\n\s*Reasoning Paragraph:", text, re.DOTALL)
    detected_biases = bias_match.group(1).strip() if bias_match else "None"

    reasoning_match = re.search(r"Reasoning Paragraph:\s*(.+?)\n\s*Bias Summary:", text, re.DOTALL)
    reasoning = reasoning_match.group(1).strip() if reasoning_match else ""

    summary_match = re.search(r"Bias Summary:\s*(.+)", text, re.DOTALL)
    summary = summary_match.group(1).strip() if summary_match else ""

    return detected_biases, reasoning, summary

# === Load your dataset ===
df = pd.read_csv("author_334_posts.csv").iloc[1000:1100].copy()
if 'input_text' not in df.columns:
    raise ValueError("❌ 'input_text' column not found in the CSV file.")

# === Initialize processing ===
results, reasons, summaries = [], [], []
calls_per_minute = 30
max_daily_calls = 1000
call_count = 0
start_time = time.time()

# === Main loop ===
for i, row in df.iterrows():
    if call_count >= max_daily_calls:
        print("🔒 Daily API limit (1000) reached. Stopping.")
        break

    if call_count > 0 and call_count % calls_per_minute == 0:
        elapsed = time.time() - start_time
        if elapsed < 60:
            time.sleep(60 - elapsed)
        start_time = time.time()

    prompt = make_prompt(row['input_text'])
    retries = 0
    while True:
        try:
            response = openai.ChatCompletion.create(
                model="meta-llama/llama-4-scout-17b-16e-instruct",
                messages=[
                    {"role": "system", "content": "You are a behavioral finance expert trained to detect financial decision-making biases."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.1
            )
            call_count += 1

            text = response['choices'][0]['message']['content'].strip()
            print(f"✅ Row {i} processed.")

            detected_biases, reasoning, summary = parse_response(text)
            results.append(detected_biases)
            reasons.append(reasoning)
            summaries.append(summary)
            break

        except openai.error.RateLimitError:
            wait = 2 ** retries
            print(f"⚠️ Rate limit hit at row {i}. Retrying in {wait} seconds...")
            time.sleep(wait)
            retries += 1
            if retries > 10:
                print(f"❌ Max retries exceeded at row {i}. Skipping.")
                results.append("None")
                reasons.append("")
                summaries.append("")
                break

    time.sleep(2)

# === Save to CSV ===
final_df = df.copy()
final_df['detected_biases'] = results
final_df['reasoning_paragraph'] = reasons
final_df['bias_summary'] = summaries
final_df.to_csv("loss_aversion_output_1000_1100.csv", index=False)
print("✅ Saved results to loss_aversion_output_1000_1100.csv")


✅ Row 1000 processed.
✅ Row 1001 processed.
✅ Row 1002 processed.
✅ Row 1003 processed.
✅ Row 1004 processed.
✅ Row 1005 processed.
✅ Row 1006 processed.
✅ Row 1007 processed.
✅ Row 1008 processed.
✅ Row 1009 processed.
✅ Row 1010 processed.
✅ Row 1011 processed.
✅ Row 1012 processed.
✅ Row 1013 processed.
✅ Row 1014 processed.
✅ Row 1015 processed.
✅ Row 1016 processed.
✅ Row 1017 processed.
✅ Row 1018 processed.
✅ Row 1019 processed.
✅ Row 1020 processed.
✅ Row 1021 processed.
✅ Row 1022 processed.
✅ Row 1023 processed.
✅ Row 1024 processed.
✅ Row 1025 processed.
✅ Row 1026 processed.
✅ Row 1027 processed.
✅ Row 1028 processed.
✅ Row 1029 processed.
✅ Row 1030 processed.
✅ Row 1031 processed.
✅ Row 1032 processed.
✅ Row 1033 processed.
✅ Row 1034 processed.
✅ Row 1035 processed.
✅ Row 1036 processed.
✅ Row 1037 processed.
✅ Row 1038 processed.
✅ Row 1039 processed.
✅ Row 1040 processed.
✅ Row 1041 processed.
✅ Row 1042 processed.
✅ Row 1043 processed.
✅ Row 1044 processed.
✅ Row 1045

In [ ]:
import pandas as pd

# Load the corresponding input slice (1000–1648 inclusive)
df = pd.read_csv("author_334_posts.csv").iloc[1000:1672].copy()

# Add your completed results
df['detected_biases'] = results
df['reasoning_paragraph'] = reasons
df['bias_summary'] = summaries

# Save to output file
df.to_csv("1611financial_bias_detected_output_1000_1672_2.csv", index=False)
print("✅ Saved completed results to 1611financial_bias_detected_output_1672-1746.csv")


✅ Saved completed results to 1611financial_bias_detected_output_1672-1746.csv


In [ ]:

from google.colab import files

In [ ]:

files.download('loss_aversion_output_1000_1100.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
system_prompt = (
    "You are a highly skilled personality psychologist specializing in text-based personality assessment. "
    "You evaluate written statements and determine the presence of each of the Big Five (OCEAN) personality traits.\n\n"
    "Your task is to return whether each trait is present in the statement based on behavioral cues. "
    "Return only 'Yes' if the trait is clearly expressed, or 'No' if it is absent or unclear.\n\n"
    "Do not provide any explanations. Only output binary values in the required format."
)


In [ ]:
def make_prompt(text):
    return (
        "Analyze the following statement and identify whether the following traits are present:\n\n"
        "Openness, Conscientiousness, Extraversion, Agreeableness, Neuroticism.\n\n"
        "Respond strictly in this format:\n"
        "Openness: Yes/No\n"
        "Conscientiousness: Yes/No\n"
        "Extraversion: Yes/No\n"
        "Agreeableness: Yes/No\n"
        "Neuroticism: Yes/No\n\n"
        f"Statement:\n{text}"
    )


In [ ]:
import pandas as pd
import openai
import time
import re

# Big Five Traits
traits = ["Openness", "Conscientiousness", "Extraversion", "Agreeableness", "Neuroticism"]

# Prompt generator
def make_prompt(text):
    return (
        "You are a personality expert. Your task is to evaluate the presence of the Big Five personality traits "
        "in the following statement.\n\n"
        "The traits are: Openness, Conscientiousness, Extraversion, Agreeableness, Neuroticism.\n\n"
        "For each trait, write either 'Yes' if the trait is present in the text, or 'No' if it is not.\n\n"
        "Strictly follow this format:\n\n"
        "Openness: <Yes/No>\n"
        "Conscientiousness: <Yes/No>\n"
        "Extraversion: <Yes/No>\n"
        "Agreeableness: <Yes/No>\n"
        "Neuroticism: <Yes/No>\n\n"
        f"Statement:\n{text}"
    )

# Parse response
def parse_response(text):
    result = {}
    for trait in traits:
        match = re.search(fr"{trait}:\s*(Yes|No)", text, re.IGNORECASE)
        result[trait] = match.group(1).capitalize() if match else "No"
    return result

# Load Data
df = pd.read_csv("author_334_posts.csv").iloc[2800:2802].copy()
if 'input_text' not in df.columns:
    raise ValueError("❌ 'input_text' column not found in the CSV file.")

# API Configuration
calls_per_minute = 30
max_daily_calls = 1000
call_count = 0
start_time = time.time()

# Prepare storage
for trait in traits:
    df[trait] = ""

# System prompt (missing in your original code)
system_prompt = "You are a personality classification assistant for the Big Five traits."

# Main Loop
for i, row in df.iterrows():
    if call_count >= max_daily_calls:
        print("🔒 Daily API limit reached. Saving partial results.")
        break

    if call_count > 0 and call_count % calls_per_minute == 0:
        elapsed = time.time() - start_time
        if elapsed < 60:
            print(f"⏳ Waiting {60 - elapsed:.2f}s to respect rate limit...")
            time.sleep(60 - elapsed)
        start_time = time.time()

    prompt = make_prompt(row['input_text'])

    retries = 0
    while True:
        try:
            response = openai.ChatCompletion.create(
                # model="meta-llama/llama-4-maverick-17b-128e-instruct",
                model="meta-llama/llama-4-scout-17b-16e-instruct",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.1
            )

            call_count += 1
            text = response['choices'][0]['message']['content'].strip()
            print(f"✅ Row {i} processed.")

            parsed = parse_response(text)
            for trait in traits:
                df.at[i, trait] = parsed[trait]

            break

        except openai.error.RateLimitError:
            wait = 2 ** retries
            print(f"⚠️ Rate limit hit at row {i}. Retrying in {wait} seconds...")
            time.sleep(wait)
            retries += 1
            if retries > 30:
                print(f"❌ Max retries exceeded at row {i}. Marking as No.")
                for trait in traits:
                    df.at[i, trait] = "No"
                break

        except Exception as e:
            print(f"❌ Unexpected error at row {i}: {e}")
            for trait in traits:
                df.at[i, trait] = "No"
            break

    time.sleep(2)

# Save progress



✅ Row 2800 processed.
✅ Row 2801 processed.


In [ ]:
# final_df = df.copy()
# for trait in traits:
#     final_df[trait] = trait_presence[trait]

# final_df.to_csv("big5_binary_presence_output_500_1014.csv", index=False)
# print("✅ Saved results to big5_binary_presence_output_0_499.csv")

# from google.colab import files
# files.download('big5_binary_presence_output_500_1014.csv')
df.to_csv("big5_binary_presence_output_2.csv", index=False)

from google.colab import files
files.download('big5_binary_presence_output_2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import openai
import time
import re

# Prompt generator
loss_aversion_keywords = {
    "Fear of loss": "Avoiding actions that could result in financial loss, even if potential gains outweigh the risks.",
    "Holding losers": "Not selling underperforming assets to avoid the pain of realizing a loss.",
    "Selling winners too early": "Selling profitable investments prematurely to avoid potential future loss.",
    "Disproportionate pain from losses": "Losses feel psychologically stronger than equivalent gains.",
    "Avoiding risky but rational trades": "Skipping profitable investments due to emotional sensitivity to possible losses."
}

def make_prompt(text):
    keyword_lines = "\n".join([f"- {k}: {v}" for k, v in loss_aversion_keywords.items()])
    return (
        "You are a behavioral finance expert.\n\n"
        "Analyze the following social media post to determine if it reflects **Loss Aversion** bias.\n"
        "Loss Aversion is only present when a user clearly describes a **personal investment decision** "
        "(such as buying, holding, or selling a financial asset) and shows **emotional discomfort** "
        "related to a potential or realized financial loss.\n\n"
        "Use the following **behavioral signs** to guide your analysis:\n"
        "- Emotional Language Around Loss (regret, fear, pain, devastated, etc.)\n"
        "- Comparative Framing (emphasizing losses over gains)\n"
        "- Risk-Averse Behavior (avoiding selling at a loss)\n"
        "- Endowment Effect (overvaluing owned assets)\n"
        "- Status Quo Bias (preferring inaction)\n"
        "- Temporal References (mentioning past loss to justify current behavior)\n\n"
        "Additional indicators of Loss Aversion include:\n"
        f"{keyword_lines}\n\n"
        "Only classify as 'Loss Aversion' if at least one of the above indicators is found "
        "**in the context of a personal financial decision**.\n\n"
        "Strictly follow this format:\n\n"
        "Loss Aversion: <Yes/No>\n"
        "Probability: <decimal value between 0 and 1>\n\n"
        "Reasoning:\n"
        "<One-paragraph explanation justifying your decision>\n\n"
        f"Statement:\n{text}"
    )
def parse_response(text):
    bias = "No"
    prob = 0.0
    reasoning = ""

    # Match Loss Aversion
    match = re.search(r"Loss Aversion:\s*(Yes|No)", text, re.IGNORECASE)
    if match:
        bias = match.group(1).capitalize()

    # Match Probability
    prob_match = re.search(r"Probability:\s*([0-9.]+)", text)
    if prob_match:
        try:
            prob = float(prob_match.group(1))
        except:
            prob = 0.0

    # Match Reasoning
    reasoning_match = re.search(r"Reasoning:\s*(.*?)(?:\n\s*Statement:|$)", text, re.DOTALL)
    if reasoning_match:
        reasoning = reasoning_match.group(1).strip()

    return bias, prob, reasoning


# Load input
df = pd.read_csv("author_334_posts.csv").iloc[1000:1025].copy()
if 'input_text' not in df.columns:
    raise ValueError("❌ 'input_text' column not found in the CSV file.")
df['Loss_Aversion'] = ""
df['Probability'] = 0.0
df['Reasoning'] = ""


# API rate limit settings
calls_per_minute = 30
max_daily_calls = 1000
call_count = 0
start_time = time.time()

# Loop
for i, row in df.iterrows():
    if call_count >= max_daily_calls:
        print("🔒 Daily API limit reached.")
        break

    if call_count > 0 and call_count % calls_per_minute == 0:
        elapsed = time.time() - start_time
        if elapsed < 60:
            print(f"⏳ Waiting {60 - elapsed:.2f}s to respect rate limit...")
            time.sleep(60 - elapsed)
        start_time = time.time()

    prompt = make_prompt(row['input_text'])

    retries = 0
    while True:
        try:
            response = openai.ChatCompletion.create(
                model="meta-llama/llama-4-scout-17b-16e-instruct",
                messages=[
                    {"role": "system", "content": "You are a behavioral finance expert trained to detect Loss Aversion bias in personal investment decisions."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.1
            )
            call_count += 1

            text = response['choices'][0]['message']['content'].strip()
            print(f"\n📤 Row {i} Response:\n{text}\n")

            loss_aversion, prob, reasoning = parse_response(text)
            df.at[i, 'Loss_Aversion'] = loss_aversion
            df.at[i, 'Probability'] = prob
            df.at[i, 'Reasoning'] = reasoning


            break

        except openai.error.RateLimitError:
            wait = 2 ** retries
            print(f"⚠️ Rate limit hit at row {i}. Retrying in {wait}s...")
            time.sleep(wait)
            retries += 1
            if retries > 10:
                print(f"❌ Max retries exceeded at row {i}. Marking as No.")
                df.at[i, 'Loss_Aversion'] = "No"
                df.at[i, 'Reasoning'] = "Rate limit exceeded."
                break

        except Exception as e:
            print(f"❌ Unexpected error at row {i}: {e}")
            df.at[i, 'Loss_Aversion'] = "No"
            df.at[i, 'Reasoning'] = f"Error: {e}"
            break

    time.sleep(2)

# Preview before save
print("\n📦 Final few rows before saving:\n", df.tail(3))

# Save output
df.to_csv("loss_aversion_new.csv", index=False)
print("✅ Saved to loss_aversion_output_with_reasoning_1000_1020.csv")



📤 Row 1000 Response:
Loss Aversion: No
Probability: 0.0

Reasoning:
The social media post discusses the potential impact of low 10-year Treasury Bill rates on bond mutual funds and ETFs, specifically the reduction in dividend yields. However, the post does not describe a personal investment decision, nor does it express emotional discomfort related to a potential or realized financial loss. The tone is informative and cautionary, rather than emotional or regretful. The author presents a hypothetical scenario and asks a rhetorical question, but does not reveal any personal actions or feelings that would indicate Loss Aversion bias. Therefore, I classify this post as not reflecting Loss Aversion bias.


📤 Row 1001 Response:
Loss Aversion: No
Probability: 0.0

Reasoning: The social media post does not reflect a personal investment decision, nor does it express emotional discomfort related to a potential or realized financial loss. The user is asking a general question about the potential

In [ ]:

files.download('loss_aversion_new.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import openai
import time
import re

# Define keywords and weights for Loss Aversion indicators
loss_aversion_keywords = {
    "Holding losses": {
        "definition": "Not selling underperforming assets to avoid the pain of realizing a loss.",
        "weight": 0.9
    },
    "Temporal references": {
        "definition": "Mentioning past losses to justify current decisions.",
        "weight": 0.9
    },
    "Comparative framing": {
        "definition": "Highlighting what was lost versus what could have been gained.",
        "weight": 0.4
    },
    "Status quo bias": {
        "definition": "Preferring inaction to avoid the possibility of loss.",
        "weight": 0.4
    },
    "Endowment effect": {
        "definition": "Overvaluing assets simply because they are owned.",
        "weight": 0.1
    },
    "Fear-related emotion": {
        "definition": "Words or tone reflecting emotional discomfort (e.g., fear, regret, anxiety).",
        "weight": 0.1
    }
}

# Prompt generator
def make_prompt(text):
    keyword_section = "\n".join([
        f"- {k}: {v['definition']} (Weight: {v['weight']})"
        for k, v in loss_aversion_keywords.items()
    ])

    instructions = (
        "You are a behavioral finance expert.\n\n"
        "Analyze the following social media post to detect **Loss Aversion** bias.\n"
        "First, check for the presence of any of the keywords below.\n"
        "Each keyword belongs to a behavioral indicator and has a weight:\n"
        f"{keyword_section}\n\n"
        "Instructions:\n"
        "- Match keywords in the post and sum their weights to calculate the score.\n"
        "- Normalize the score between 0 and 1.\n"
        "- If the normalized score is greater than 0.5, classify as 'Yes'.\n"
        "- Otherwise, classify as 'No'.\n"
        "- Set both Probability and Confidence equal to the normalized score.\n\n"
        "Respond in this exact format:\n"
        "Loss Aversion: <Yes/No>\n"
        "Probability: <decimal between 0 and 1>\n"
        "Confidence: <decimal between 0 and 1>\n\n"
        "Reasoning:\n<One-paragraph explanation, mention matched keywords and score>\n\n"
        f"Statement:\n{text}"
    )
    return instructions

# Parse model output
def parse_response(text):
    bias = "No"
    prob = 0.0
    confidence = 0.0
    reasoning = ""

    match = re.search(r"Loss Aversion:\s*(Yes|No)", text, re.IGNORECASE)
    if match:
        bias = match.group(1).capitalize()

    prob_match = re.search(r"Probability:\s*([0-9.]+)", text)
    if prob_match:
        try:
            prob = float(prob_match.group(1))
        except:
            prob = 0.0

    conf_match = re.search(r"Confidence:\s*([0-9.]+)", text)
    if conf_match:
        try:
            confidence = float(conf_match.group(1))
        except:
            confidence = 0.0

    reasoning_match = re.search(r"Reasoning:\s*(.*?)(?:\n\s*Statement:|$)", text, re.DOTALL)
    if reasoning_match:
        reasoning = reasoning_match.group(1).strip()

    return bias, prob, confidence, reasoning

# Load input
df = pd.read_csv("author_334_posts.csv").iloc[1000:1025].copy()
if 'input_text' not in df.columns:
    raise ValueError("❌ 'input_text' column not found in the CSV file.")

df['Loss_Aversion'] = ""
df['Probability'] = 0.0
df['Confidence'] = 0.0
df['Reasoning'] = ""

# API rate limit settings
calls_per_minute = 30
max_daily_calls = 1000
call_count = 0
start_time = time.time()

# Loop through posts
for i, row in df.iterrows():
    if call_count >= max_daily_calls:
        print("🔒 Daily API limit reached.")
        break

    if call_count > 0 and call_count % calls_per_minute == 0:
        elapsed = time.time() - start_time
        if elapsed < 60:
            print(f"⏳ Waiting {60 - elapsed:.2f}s to respect rate limit...")
            time.sleep(60 - elapsed)
        start_time = time.time()

    prompt = make_prompt(row['input_text'])

    retries = 0
    while True:
        try:
            response = openai.ChatCompletion.create(
                model="meta-llama/llama-4-scout-17b-16e-instruct",
                messages=[
                    {"role": "system", "content": "You are a behavioral finance expert trained to detect Loss Aversion bias in personal investment decisions."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.1
            )
            call_count += 1

            text = response['choices'][0]['message']['content'].strip()
            print(f"\n📤 Row {i} Response:\n{text}\n")

            loss_aversion, prob, confidence, reasoning = parse_response(text)
            df.at[i, 'Loss_Aversion'] = loss_aversion
            df.at[i, 'Probability'] = prob
            df.at[i, 'Confidence'] = confidence
            df.at[i, 'Reasoning'] = reasoning
            break

        except openai.error.RateLimitError:
            wait = 2 ** retries
            print(f"⚠️ Rate limit hit at row {i}. Retrying in {wait}s...")
            time.sleep(wait)
            retries += 1
            if retries > 10:
                print(f"❌ Max retries exceeded at row {i}. Marking as No.")
                df.at[i, 'Loss_Aversion'] = "No"
                df.at[i, 'Reasoning'] = "Rate limit exceeded."
                break

        except Exception as e:
            print(f"❌ Unexpected error at row {i}: {e}")
            df.at[i, 'Loss_Aversion'] = "No"
            df.at[i, 'Reasoning'] = f"Error: {e}"
            break

    time.sleep(2)

# Save output
df.to_csv("loss_aversion_with_confidence.csv", index=False)
print("✅ Saved to loss_aversion_with_confidence.csv")



📤 Row 1000 Response:
Loss Aversion: Yes
Probability: 0.72
Confidence: 0.72

Reasoning: 
The social media post expresses concerns about potential losses in Bond Mutual Funds and ETFs due to the low dividend yields caused by the record low 10-year Treasury Bill rates. The keywords that match are "Holding losses" (implied, as the author is worried about not selling or adjusting their portfolios to avoid losses, weight: 0.9) and "Fear-related emotion" (implied, as the author seems anxious about the potential drop in dividend yields, weight: 0.1). Additionally, the mention of "Intermediate Bond Funds seem scary now" implies a sense of unease, further supporting the presence of Loss Aversion. The total weight is 1.0 (0.9 + 0.1). Normalizing this score between 0 and 1, we get 1.0. However, to follow a more conservative approach and given that we have only two indicators, the normalized score is 0.72 (as  (0.9+0.1)/ (0.9+0.1+ other weights if any present), but weights of other indicators are 

In [ ]:

files.download('loss_aversion_with_confidence.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import openai
import time
import re

# Define keywords and weights for Loss Aversion indicators
loss_aversion_keywords = {
    "Holding losses": {
        "definition": "Not selling underperforming assets to avoid the pain of realizing a loss.",
        "weight": 0.7
    },
    "Temporal references": {
        "definition": "Mentioning past losses to justify current decisions.",
        "weight": 0.45
    },
    "Comparative framing": {
        "definition": "Highlighting what was lost versus what could have been gained.",
        "weight": 0.40
    },
    "Status quo bias": {
        "definition": "Preferring inaction to avoid the possibility of loss.",
        "weight": 0.15
    },
    "Endowment effect": {
        "definition": "Overvaluing assets simply because they are owned.",
        "weight": 0.15
    },
    "Fear-related emotion": {
        "definition": "Words or tone reflecting emotional discomfort (e.g., fear, regret, anxiety).",
        "weight": 0.1
    }
}

# Prompt generator

def make_prompt(text):
    keyword_section = "\n".join([f"- {k}: {v['definition']} (Weight: {v['weight']})" for k, v in loss_aversion_keywords.items()])
    return (
        "You are a behavioral finance expert.\n\n"
        "Analyze the following social media post to determine if it reflects **Loss Aversion** bias.\n"
        "Loss Aversion is only present when a user clearly describes a **personal investment decision**\n"
        "(such as buying, holding, or selling a financial asset) and shows **emotional discomfort**\n"
        "related to a potential or realized financial loss.\n\n"
        "Use the following **behavioral signs**, weighted by relevance, to guide your decision.\n"
        f"{keyword_section}\n\n"
        "Only classify as 'Loss Aversion' if at least one of the above indicators is found\n"
        "**in the context of a personal financial decision**.\n\n"
        "Base your `Probability` and `Confidence` on the presence and weight of these indicators.\n"
        "The more high-weight indicators you find, the higher both values should be.\n\n"
        "Strictly follow this format:\n\n"
        "Loss Aversion: <Yes/No>\n"
        "Probability: <decimal value between 0 and 1>\n"
        "Confidence: <decimal value between 0 and 1>\n\n"
        "Reasoning:\n"
        "<One-paragraph explanation justifying your decision>\n\n"
        f"Statement:\n{text}"
    )

# Parse model output

def parse_response(text):
    bias = "No"
    prob = 0.0
    confidence = 0.0
    reasoning = ""

    match = re.search(r"Loss Aversion:\s*(Yes|No)", text, re.IGNORECASE)
    if match:
        bias = match.group(1).capitalize()

    prob_match = re.search(r"Probability:\s*([0-9.]+)", text)
    if prob_match:
        try:
            prob = float(prob_match.group(1))
        except:
            prob = 0.0

    conf_match = re.search(r"Confidence:\s*([0-9.]+)", text)
    if conf_match:
        try:
            confidence = float(conf_match.group(1))
        except:
            confidence = 0.0

    reasoning_match = re.search(r"Reasoning:\s*(.*?)(?:\n\s*Statement:|$)", text, re.DOTALL)
    if reasoning_match:
        reasoning = reasoning_match.group(1).strip()

    return bias, prob, confidence, reasoning

# Load input

df = pd.read_csv("author_334_posts.csv").iloc[:00].copy()
if 'input_text' not in df.columns:
    raise ValueError("❌ 'input_text' column not found in the CSV file.")

df['Loss_Aversion'] = ""
df['Probability'] = 0.0
df['Confidence'] = 0.0
df['Reasoning'] = ""

# API rate limit settings
calls_per_minute = 30
max_daily_calls = 1000
call_count = 0
start_time = time.time()

# Loop
for i, row in df.iterrows():
    if call_count >= max_daily_calls:
        print("🔒 Daily API limit reached.")
        break

    if call_count > 0 and call_count % calls_per_minute == 0:
        elapsed = time.time() - start_time
        if elapsed < 60:
            print(f"⏳ Waiting {60 - elapsed:.2f}s to respect rate limit...")
            time.sleep(60 - elapsed)
        start_time = time.time()

    prompt = make_prompt(row['input_text'])

    retries = 0
    while True:
        try:
            response = openai.ChatCompletion.create(
                model="meta-llama/llama-4-scout-17b-16e-instruct",
                messages=[
                    {"role": "system", "content": "You are a behavioral finance expert trained to detect Loss Aversion bias in personal investment decisions."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.1
            )
            call_count += 1

            text = response['choices'][0]['message']['content'].strip()
            print(f"\n📤 Row {i} Response:\n{text}\n")

            loss_aversion, prob, confidence, reasoning = parse_response(text)
            df.at[i, 'Loss_Aversion'] = loss_aversion
            df.at[i, 'Probability'] = prob
            df.at[i, 'Confidence'] = confidence
            df.at[i, 'Reasoning'] = reasoning
            break

        except openai.error.RateLimitError:
            wait = 2 ** retries
            print(f"⚠️ Rate limit hit at row {i}. Retrying in {wait}s...")
            time.sleep(wait)
            retries += 1
            if retries > 10:
                print(f"❌ Max retries exceeded at row {i}. Marking as No.")
                df.at[i, 'Loss_Aversion'] = "No"
                df.at[i, 'Reasoning'] = "Rate limit exceeded."
                break

        except Exception as e:
            print(f"❌ Unexpected error at row {i}: {e}")
            df.at[i, 'Loss_Aversion'] = "No"
            df.at[i, 'Reasoning'] = f"Error: {e}"
            break

    time.sleep(2)

# Save output
df.to_csv("loss_aversion_with_confidence.csv", index=False)
print("✅ Saved to loss_aversion_with_confidence.csv")



📤 Row 0 Response:
Loss Aversion: No
Probability: 0.0
Confidence:  

The statement describes information<|header_start|> aboutassistant<|header_start|>

Loss Aon: No
 
Probability 
0;00 
 
assistant<

 I 

Loss Aversion: No
Probability: <|header_start|><|header_start|>assistant<

0.0
Confidence: 0.0

Reasoning: The statement does not describe a personal investment decision, nor does it mention any emotional discomfort related to a potential or realized financial loss. The individual is simply expressing frustration and curiosity about their credit union's policy regarding ATM deposits, specifically the hold on funds deposited in cash. There is no indication of Loss Aversion bias or any of the specified behavioral signs.


📤 Row 1 Response:
Loss Aversion: No
Probability: 0.0
Confidence: 0.0

Reasoning:
The social media post does not reflect a personal investment decision, but rather a complaint about a banking issue with American Express. The user is discussing a problem with a check be

In [ ]:

files.download('loss_aversion_with_confidence.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import openai
import time
import re

# Define keywords and weights for Loss Aversion indicators
loss_aversion_keywords = {
    "Holding losses": {
        "definition": "Not selling underperforming assets to avoid the pain of realizing a loss.",
        "weight": 0.9,
        "keywords": [
            "still holding", "bagholding", "can't sell", "won't sell", "refuse to sell",
            "not cutting losses", "holding despite loss"
        ]
    },
    "Temporal references": {
        "definition": "Mentioning past losses to justify current decisions.",
        "weight": 0.9,
        "keywords": [
            "since I lost", "after the last crash", "I lost money before"
        ]
    },
    "Comparative framing": {
        "definition": "Highlighting what was lost versus what could have been gained.",
        "weight": 0.4,
        "keywords": [
            "could have made", "should have sold", "missed the gains"
        ]
    },
    "Status quo bias": {
        "definition": "Preferring inaction to avoid the possibility of loss.",
        "weight": 0.4,
        "keywords": [
            "better to wait", "doing nothing is safer"
        ]
    },
    "Endowment effect": {
        "definition": "Overvaluing assets simply because they are owned.",
        "weight": 0.1,
        "keywords": [
            "feels wrong to sell", "personal attachment"
        ]
    },
    "Fear-related emotion": {
        "definition": "Words or tone reflecting emotional discomfort (e.g., fear, regret, anxiety).",
        "weight": 0.1,
        "keywords": [
            "regret", "scared", "nervous", "anxious", "panicking", "stressed", "worried"
        ]
    }
}

# Prompt generator
def make_prompt(text):
    keyword_section = ""
    for name, data in loss_aversion_keywords.items():
        kws = ", ".join(f'"{kw}"' for kw in data.get("keywords", []))
        keyword_section += f"- {name}: {data['definition']} (Weight: {data['weight']})\n  Keywords: {kws}\n"

    instructions = (
        "You are a behavioral finance expert.\n\n"
        "Analyze the following social media post to detect **Loss Aversion** bias.\n"
        "First, check for the presence of any of the keywords below.\n"
        "Each keyword belongs to a behavioral indicator and has a weight:\n"
        f"{keyword_section}\n\n"
        "Instructions:\n"
        "- Match keywords in the post and sum their weights to calculate the score.\n"
        "- Normalize the score between 0 and 1.\n"
        "- If the normalized score is greater than 0.5, classify as 'Yes'.\n"
        "- Otherwise, classify as 'No'.\n"
        "- Set both Probability and Confidence equal to the normalized score.\n\n"
        "Respond in this exact format:\n"
        "Loss Aversion: <Yes/No>\n"
        "Probability: <decimal between 0 and 1>\n"
        "Confidence: <decimal between 0 and 1>\n\n"
        "Reasoning:\n<One-paragraph explanation, mention matched keywords and score>\n\n"
        f"Statement:\n{text}"
    )
    return instructions

# The rest of the code (unchanged)
def parse_response(text):
    bias = "No"
    prob = 0.0
    confidence = 0.0
    reasoning = ""

    match = re.search(r"Loss Aversion:\s*(Yes|No)", text, re.IGNORECASE)
    if match:
        bias = match.group(1).capitalize()

    prob_match = re.search(r"Probability:\s*([0-9.]+)", text)
    if prob_match:
        try:
            prob = float(prob_match.group(1))
        except:
            prob = 0.0

    conf_match = re.search(r"Confidence:\s*([0-9.]+)", text)
    if conf_match:
        try:
            confidence = float(conf_match.group(1))
        except:
            confidence = 0.0

    reasoning_match = re.search(r"Reasoning:\s*(.*?)(?:\n\s*Statement:|$)", text, re.DOTALL)
    if reasoning_match:
        reasoning = reasoning_match.group(1).strip()

    return bias, prob, confidence, reasoning

# Load input
df = pd.read_csv("author_334_posts.csv").iloc[1000:1125].copy()
if 'input_text' not in df.columns:
    raise ValueError("❌ 'input_text' column not found in the CSV file.")

df['Loss_Aversion'] = ""
df['Probability'] = 0.0
df['Confidence'] = 0.0
df['Reasoning'] = ""

# API rate limit settings
calls_per_minute = 30
max_daily_calls = 1000
call_count = 0
start_time = time.time()

# Loop through posts
for i, row in df.iterrows():
    if call_count >= max_daily_calls:
        print("🔒 Daily API limit reached.")
        break

    if call_count > 0 and call_count % calls_per_minute == 0:
        elapsed = time.time() - start_time
        if elapsed < 60:
            print(f"⏳ Waiting {60 - elapsed:.2f}s to respect rate limit...")
            time.sleep(60 - elapsed)
        start_time = time.time()

    prompt = make_prompt(row['input_text'])

    retries = 0
    while True:
        try:
            response = openai.ChatCompletion.create(
                model="meta-llama/llama-4-scout-17b-16e-instruct",
                messages=[
                    {"role": "system", "content": "You are a behavioral finance expert trained to detect Loss Aversion bias in personal investment decisions."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.1
            )
            call_count += 1

            text = response['choices'][0]['message']['content'].strip()
            print(f"\n📤 Row {i} Response:\n{text}\n")

            loss_aversion, prob, confidence, reasoning = parse_response(text)
            df.at[i, 'Loss_Aversion'] = loss_aversion
            df.at[i, 'Probability'] = prob
            df.at[i, 'Confidence'] = confidence
            df.at[i, 'Reasoning'] = reasoning
            break

        except openai.error.RateLimitError:
            wait = 2 ** retries
            print(f"⚠️ Rate limit hit at row {i}. Retrying in {wait}s...")
            time.sleep(wait)
            retries += 1
            if retries > 10:
                print(f"❌ Max retries exceeded at row {i}. Marking as No.")
                df.at[i, 'Loss_Aversion'] = "No"
                df.at[i, 'Reasoning'] = "Rate limit exceeded."
                break

        except Exception as e:
            print(f"❌ Unexpected error at row {i}: {e}")
            df.at[i, 'Loss_Aversion'] = "No"
            df.at[i, 'Reasoning'] = f"Error: {e}"
            break

    time.sleep(2)

# Save output
df.to_csv("loss_aversion_with_confidence.csv", index=False)
print("✅ Saved to loss_aversion_with_confidence.csv")



📤 Row 1000 Response:
Loss Aversion: No
Probability: 0.225
Confidence: 0.225

Reasoning: 
The social media post expresses concern about the potential impact of low Treasury Bill rates on Bond Funds, particularly for retired individuals who have a significant allocation to these funds. Although the post mentions that Intermediate Bond Funds "seem scary now", this emotional tone (Weight: 0.1) only slightly contributes to the overall score. No direct matches were found for other keywords related to Loss Aversion, such as "holding losses", "temporal references", "comparative framing", "status quo bias", or "endowment effect". The post primarily presents a factual analysis of the situation and its potential consequences, rather than a biased decision-making process driven by Loss Aversion. The total score is 0.1, which when normalized, equals 0.225, below the threshold of 0.5.


📤 Row 1001 Response:
Loss Aversion: No
Probability: 0.0
Confidence: 0.0

Reasoning: 
The social media post does n

In [ ]:
# Updated full code with model-assigned qualitative weight logic

import pandas as pd
import openai
import time
import re

# Define keywords and their behavioral categories (no numeric weights now)
loss_aversion_keywords = {
    "Holding losses": {
        "definition": "Not selling underperforming assets to avoid the pain of realizing a loss.",
        "keywords": [
            "still holding", "bagholding", "can't sell", "won't sell", "refuse to sell",
            "not cutting losses", "holding despite loss"
        ]
    },
    "Temporal references": {
        "definition": "Mentioning past losses to justify current decisions.",
        "keywords": [
            "since I lost", "after the last crash", "I lost money before"
        ]
    },
    "Comparative framing": {
        "definition": "Highlighting what was lost versus what could have been gained.",
        "keywords": [
            "could have made", "should have sold", "missed the gains"
        ]
    },
    "Status quo bias": {
        "definition": "Preferring inaction to avoid the possibility of loss.",
        "keywords": [
            "better to wait", "doing nothing is safer"
        ]
    },
    "Endowment effect": {
        "definition": "Overvaluing assets simply because they are owned.",
        "keywords": [
            "feels wrong to sell", "personal attachment"
        ]
    },
    "Fear-related emotion": {
        "definition": "Words or tone reflecting emotional discomfort (e.g., fear, regret, anxiety).",
        "keywords": [
            "regret", "scared", "nervous", "anxious", "panicking", "stressed", "worried"
        ]
    }
}

# Prompt generator
def make_prompt(text):
    keyword_section = ""
    for name, data in loss_aversion_keywords.items():
        kws = ", ".join(f'"{kw}"' for kw in data["keywords"])
        keyword_section += f"- {name}: {data['definition']}\n  Keywords: {kws}\n"

    instructions = (
        "You are a behavioral finance expert.\n\n"
        "Your task is to analyze a social media post and determine whether it reflects **Loss Aversion** bias.\n"
        "Loss Aversion refers to emotional discomfort in response to a potential or realized financial loss, especially when making a **personal investment decision** (e.g., buying, holding, selling).\n\n"
        "**Behavioral Indicators and Keywords**:\n"
        f"{keyword_section}\n\n"
        "**Instructions**:\n"
        "- Identify all matched keywords in the post.\n"
        "- For each matched keyword, assign a **qualitative weight**: **High**, **Mid**, or **Low**, based on how clearly and strongly it reflects Loss Aversion **in the context of the user's financial decision**.\n"
        "- Ignore keywords if they are used outside the context of a personal financial decision.\n"
        "- If one or more matched keywords are rated **High or Mid**, then set `Loss Aversion: Yes`. Else, say `No`.\n"
        "- Set both **Probability** and **Confidence** based on your interpretation of how strong the indicators are:\n"
        "   • Mostly High: 0.8–1.0\n"
        "   • Mostly Mid: 0.5–0.7\n"
        "   • Mostly Low or unclear: ≤ 0.4\n\n"
        "**Respond in this format**:\n"
        "Loss Aversion: <Yes/No>\n"
        "Probability: <decimal from 0 to 1>\n"
        "Confidence: <decimal from 0 to 1>\n\n"
        "Reasoning:\n"
        "- List matched keywords and which trait they belong to.\n"
        "- For each, specify the qualitative weight (High/Mid/Low) **and your rationale** for assigning that weight.\n"
        "- Conclude with a short judgment on why the post is or isn't showing Loss Aversion.\n\n"
        f"Statement:\n{text}"
    )

    return instructions

# Response parser
def parse_response(text):
    bias = "No"
    prob = 0.0
    confidence = 0.0
    reasoning = ""

    match = re.search(r"Loss Aversion:\s*(Yes|No)", text, re.IGNORECASE)
    if match:
        bias = match.group(1).capitalize()

    prob_match = re.search(r"Probability:\s*([0-9.]+)", text)
    if prob_match:
        try:
            prob = float(prob_match.group(1))
        except:
            prob = 0.0

    conf_match = re.search(r"Confidence:\s*([0-9.]+)", text)
    if conf_match:
        try:
            confidence = float(conf_match.group(1))
        except:
            confidence = 0.0

    reasoning_match = re.search(r"Reasoning:\s*(.*?)(?:\n\s*Statement:|$)", text, re.DOTALL)
    if reasoning_match:
        reasoning = reasoning_match.group(1).strip()

    return bias, prob, confidence, reasoning

# Load input CSV
df = pd.read_csv("author_334_posts.csv").iloc[1000:1125].copy()
if 'input_text' not in df.columns:
    raise ValueError("❌ 'input_text' column not found in the CSV file.")

df['Loss_Aversion'] = ""
df['Probability'] = 0.0
df['Confidence'] = 0.0
df['Reasoning'] = ""

# API rate limit settings
calls_per_minute = 30
max_daily_calls = 1000
call_count = 0
start_time = time.time()

# Main loop
for i, row in df.iterrows():
    if call_count >= max_daily_calls:
        print("🔒 Daily API limit reached.")
        break

    if call_count > 0 and call_count % calls_per_minute == 0:
        elapsed = time.time() - start_time
        if elapsed < 60:
            print(f"⏳ Waiting {60 - elapsed:.2f}s to respect rate limit...")
            time.sleep(60 - elapsed)
        start_time = time.time()

    prompt = make_prompt(row['input_text'])

    retries = 0
    while True:
        try:
            response = openai.ChatCompletion.create(
                model="meta-llama/llama-4-scout-17b-16e-instruct",
                messages=[
                    {"role": "system", "content": "You are a behavioral finance expert trained to detect Loss Aversion bias in personal investment decisions."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.1
            )
            call_count += 1

            text = response['choices'][0]['message']['content'].strip()
            print(f"\n📤 Row {i} Response:\n{text}\n")

            loss_aversion, prob, confidence, reasoning = parse_response(text)
            df.at[i, 'Loss_Aversion'] = loss_aversion
            df.at[i, 'Probability'] = prob
            df.at[i, 'Confidence'] = confidence
            df.at[i, 'Reasoning'] = reasoning
            break

        except openai.error.RateLimitError:
            wait = 2 ** retries
            print(f"⚠️ Rate limit hit at row {i}. Retrying in {wait}s...")
            time.sleep(wait)
            retries += 1
            if retries > 10:
                print(f"❌ Max retries exceeded at row {i}. Marking as No.")
                df.at[i, 'Loss_Aversion'] = "No"
                df.at[i, 'Reasoning'] = "Rate limit exceeded."
                break

        except Exception as e:
            print(f"❌ Unexpected error at row {i}: {e}")
            df.at[i, 'Loss_Aversion'] = "No"
            df.at[i, 'Reasoning'] = f"Error: {e}"
            break

    time.sleep(2)

# Save output
df.to_csv("loss_aversion_with_confidence.csv", index=False)
print("✅ Saved to loss_aversion_with_confidence.csv")




📤 Row 1000 Response:
Loss Aversion: No
Probability: 0.2
Confidence: 0.3

Reasoning:

The post does not explicitly exhibit strong indicators of Loss Aversion. However, some keywords and phrases could be interpreted as reflecting concerns about potential losses.

- "Intermediate Bond Funds seem scary now" could indicate a **fear-related emotion** (Low weight). The term "scary" reflects some level of emotional discomfort or anxiety about the potential performance of Intermediate Bond Funds.
- "hurt returns" might imply a concern about potential losses (Low weight). This phrase indicates a worry about the impact on returns but does not directly express an aversion to realizing losses.

No matched keywords were rated **High** or **Mid**. The discussion primarily focuses on the expected performance of bond funds and the implications of low Treasury Bill rates on dividend yields, rather than a personal investment decision driven by Loss Aversion.

The overall tone is more concerned and analy

In [ ]:

files.download('loss_aversion_with_confidence.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import openai
import time
import re

loss_aversion_keywords = {
    "Holding losses": {"definition": "Not selling underperforming assets to avoid the pain of realizing a loss."},
    "Temporal references": {"definition": "Mentioning past losses to justify current decisions."},
    "Comparative framing": {"definition": "Highlighting what was lost versus what could have been gained."},
    "Status quo bias": {"definition": "Preferring inaction to avoid the possibility of loss."},
    "Endowment effect": {"definition": "Overvaluing assets simply because they are owned."},
    "Fear-related emotion": {"definition": "Words or tone reflecting emotional discomfort (e.g., fear, regret, anxiety)."}
}

def make_prompt(text):
    trait_definitions = ""
    for name, data in loss_aversion_keywords.items():
        trait_definitions += f"- {name}: {data['definition']}\n"

    return (
        "You are a behavioral finance expert.\n\n"
        "Your task is to analyze a social media post for **Loss Aversion** bias in personal financial decisions from below characterstics.\n\n"
        "**Characteristics**:\n"
        f"{trait_definitions}\n"
        "**Instructions**:\n"
        "- Identify all characteristics present in the post.\n"
        "- Assign a numeric weight (0 to 1) to each.\n"
        "- If at least one characteristic is present, set Loss Aversion: Yes. Otherwise, No.\n"
        "- Also specify:\n"
        "   • Intensity: High / Mid / Low\n\n"
        "**Respond in this format**:\n"
        "Loss Aversion: <Yes/No>\n"
        "Intensity: <High/Mid/Low>\n"
        "Reasoning:\n"
        "- List traits, weights, and rationale.\n\n"
        f"Statement:\n{text}"
    )

def parse_response(text):
    def extract(pattern):
        match = re.search(pattern, text, re.IGNORECASE)
        return match.group(1).strip() if match else ""

    return {
        'Loss_Aversion': extract(r"Loss Aversion:\s*(Yes|No)").capitalize(),
        'Intensity': extract(r"Intensity:\s*(High|Mid|Low)").capitalize(),
        'Reasoning': extract(r"Reasoning:\s*(.*)")
    }

df = pd.read_csv("author_334_posts.csv").iloc[1000:1025].copy()
df[['Loss_Aversion', 'Intensity', 'Reasoning']] = ""

calls_per_minute, max_daily_calls, call_count = 30, 1000, 0
start_time = time.time()

for i, row in df.iterrows():
    if call_count >= max_daily_calls:
        break
    if call_count > 0 and call_count % calls_per_minute == 0:
        elapsed = time.time() - start_time
        if elapsed < 60: time.sleep(60 - elapsed)
        start_time = time.time()

    prompt = make_prompt(row['input_text'])
    retries = 0

    while True:
        try:
            response = openai.ChatCompletion.create(
                model="meta-llama/llama-4-scout-17b-16e-instruct",
                messages=[
                    {"role": "system", "content": "You are a behavioral finance expert."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.1
            )
            call_count += 1
            output = parse_response(response['choices'][0]['message']['content'])
            for key in output:
                df.at[i, key] = output[key]
            break
        except Exception as e:
            if retries > 3:
                df.at[i, 'Loss_Aversion'] = "No"
                df.at[i, 'Reasoning'] = f"Error: {e}"
                break
            time.sleep(2 ** retries)
            retries += 1

    time.sleep(2)

df.to_csv("experiment1_traits_only.csv", index=False)


In [ ]:
import pandas as pd
import openai
import time
import re

loss_aversion_keywords = {
    "Holding losses": {
        "definition": "Not selling underperforming assets to avoid the pain of realizing a loss.",
        "keywords": ["still holding", "bagholding", "can't sell", "won't sell", "refuse to sell"]
    },
    "Temporal references": {
        "definition": "Mentioning past losses to justify current decisions.",
        "keywords": ["since I lost", "after the last crash"]
    },
    "Comparative framing": {
        "definition": "Highlighting what was lost versus what could have been gained.",
        "keywords": ["could have made", "should have sold", "missed the gains"]
    },
    "Status quo bias": {
        "definition": "Preferring inaction to avoid the possibility of loss.",
        "keywords": ["better to wait", "doing nothing is safer"]
    },
    "Endowment effect": {
        "definition": "Overvaluing assets simply because they are owned.",
        "keywords": ["feels wrong to sell", "personal attachment"]
    },
    "Fear-related emotion": {
        "definition": "Words or tone reflecting emotional discomfort.",
        "keywords": ["regret", "scared", "nervous", "anxious", "panicking", "worried"]
    }
}

def make_prompt(text):
    trait_definitions = ""
    for name, data in loss_aversion_keywords.items():
        keywords = ", ".join(f'"{kw}"' for kw in data['keywords'])
        trait_definitions += f"- {name}: {data['definition']}\n  Keywords: {keywords}\n"

    return (
        "You are a behavioral finance expert.\n\n"
        "Your task is to analyze a social media post for **Loss Aversion** bias in personal financial decisions.\n\n"
        "**Behavioral Characteristics with Keywords**:\n"
        f"{trait_definitions}\n"
        "**Instructions**:\n"
        "- Match keywords used in context.\n"
        "- Assign a numeric weight (0 to 1) to each trait based on presence.\n"
        "- If at least one trait is present, set Loss Aversion: Yes. Otherwise, No.\n"
        "- Also specify:\n"
        "   • Intensity: High / Mid / Low\n\n"
        "**Respond in this format**:\n"
        "Loss Aversion: <Yes/No>\n"
        "Intensity: <High/Mid/Low>\n"
        "Reasoning:\n"
        "- List traits, weights, matched keywords, and rationale.\n\n"
        f"Statement:\n{text}"
    )

def parse_response(text):
    def extract(pattern):
        match = re.search(pattern, text, re.IGNORECASE)
        return match.group(1).strip() if match else ""

    return {
        'Loss_Aversion': extract(r"Loss Aversion:\s*(Yes|No)").capitalize(),
        'Intensity': extract(r"Intensity:\s*(High|Mid|Low)").capitalize(),
        'Reasoning': extract(r"Reasoning:\s*(.*)")
    }

df = pd.read_csv("author_334_posts.csv").iloc[1000:1025].copy()
df[['Loss_Aversion', 'Intensity', 'Reasoning']] = ""

calls_per_minute, max_daily_calls, call_count = 30, 1000, 0
start_time = time.time()

for i, row in df.iterrows():
    if call_count >= max_daily_calls:
        break
    if call_count > 0 and call_count % calls_per_minute == 0:
        elapsed = time.time() - start_time
        if elapsed < 60: time.sleep(60 - elapsed)
        start_time = time.time()

    prompt = make_prompt(row['input_text'])
    retries = 0

    while True:
        try:
            response = openai.ChatCompletion.create(
                model="meta-llama/llama-4-scout-17b-16e-instruct",
                messages=[
                    {"role": "system", "content": "You are a behavioral finance expert."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.1
            )
            call_count += 1
            output = parse_response(response['choices'][0]['message']['content'])
            for key in output:
                df.at[i, key] = output[key]
            break
        except Exception as e:
            if retries > 3:
                df.at[i, 'Loss_Aversion'] = "No"
                df.at[i, 'Reasoning'] = f"Error: {e}"
                break
            time.sleep(2 ** retries)
            retries += 1

    time.sleep(2)

df.to_csv("experiment2_traits_only.csv", index=False)


In [ ]:
import pandas as pd
import openai
import time
import re

# Same keywords as Experiment 2
loss_aversion_keywords = {
    "Holding losses": {
        "definition": "Not selling underperforming assets to avoid the pain of realizing a loss.",
        "keywords": ["still holding", "bagholding", "can't sell", "won't sell", "refuse to sell"]
    },
    "Temporal references": {
        "definition": "Mentioning past losses to justify current decisions.",
        "keywords": ["since I lost", "after the last crash"]
    },
    "Comparative framing": {
        "definition": "Highlighting what was lost versus what could have been gained.",
        "keywords": ["could have made", "should have sold", "missed the gains"]
    },
    "Status quo bias": {
        "definition": "Preferring inaction to avoid the possibility of loss.",
        "keywords": ["better to wait", "doing nothing is safer"]
    },
    "Endowment effect": {
        "definition": "Overvaluing assets simply because they are owned.",
        "keywords": ["feels wrong to sell", "personal attachment"]
    },
    "Fear-related emotion": {
        "definition": "Emotional discomfort (fear, regret, anxiety).",
        "keywords": ["regret", "scared", "nervous", "anxious", "panicking", "worried"]
    }
}

def make_prompt(text):
    trait_definitions = ""
    for name, data in loss_aversion_keywords.items():
        keywords = ", ".join(f'"{kw}"' for kw in data['keywords'])
        trait_definitions += f"- {name}: {data['definition']}\n  Keywords: {keywords}\n"

    return (
        "You are a behavioral finance expert.\n\n"
        "Analyze the social media post for **Loss Aversion** bias.\n"
        "In addition to given keywords, also match semantically similar expressions.\n\n"
        "**Traits and Keywords**:\n"
        f"{trait_definitions}\n"
        "**Instructions**:\n"
        "- Use exact or semantically similar expressions to detect traits.\n"
        "- Assign numeric weights to matched traits.\n"
        "- Determine:\n"
        "   • Loss Aversion: Yes/No\n"
        "   • Intensity: High/Mid/Low\n\n"
        "**Respond in this format**:\n"
        "Loss Aversion: <Yes/No>\n"
        "Intensity: <High/Mid/Low>\n"
        "Reasoning:\n"
        "- List traits, matched (or similar) keywords, weights, and rationale.\n\n"
        f"Statement:\n{text}"
    )

def parse_response(text):
    def extract(pattern):
        match = re.search(pattern, text, re.IGNORECASE)
        return match.group(1).strip() if match else ""

    return {
        'Loss_Aversion': extract(r"Loss Aversion:\s*(Yes|No)").capitalize(),
        'Intensity': extract(r"Intensity:\s*(High|Mid|Low)").capitalize(),
        'Reasoning': extract(r"Reasoning:\s*(.*)")
    }

df = pd.read_csv("author_334_posts.csv").iloc[1000:1025].copy()
df[['Loss_Aversion', 'Intensity', 'Reasoning']] = ""


calls_per_minute, max_daily_calls, call_count = 30, 1000, 0
start_time = time.time()

for i, row in df.iterrows():
    if call_count >= max_daily_calls:
        break
    if call_count > 0 and call_count % calls_per_minute == 0:
        elapsed = time.time() - start_time
        if elapsed < 60: time.sleep(60 - elapsed)
        start_time = time.time()

    prompt = make_prompt(row['input_text'])
    retries = 0

    while True:
        try:
            response = openai.ChatCompletion.create(
                model="meta-llama/llama-4-scout-17b-16e-instruct",
                messages=[
                    {"role": "system", "content": "You are a behavioral finance expert."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.1
            )
            call_count += 1
            output = parse_response(response['choices'][0]['message']['content'])
            for key in output:
                df.at[i, key] = output[key]
            break
        except Exception as e:
            if retries > 3:
                df.at[i, 'Loss_Aversion'] = "No"
                df.at[i, 'Reasoning'] = f"Error: {e}"
                break
            time.sleep(2 ** retries)
            retries += 1

    time.sleep(2)

df.to_csv("experiment3_traits_only.csv", index=False)



In [ ]:

files.download('experiment1_traits_only.csv')
files.download('experiment2_traits_only.csv')
files.download('experiment3_traits_only.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import openai
import time
import re

# Define traits (no keywords in Experiment 1)
loss_aversion_traits = {
    "Holding losses": "Not selling underperforming assets to avoid the pain of realizing a loss.",
    "Temporal references": "Mentioning past losses to justify current decisions.",
    "Comparative framing": "Highlighting what was lost versus what could have been gained.",
    "Status quo bias": "Preferring inaction to avoid the possibility of loss.",
    "Endowment effect": "Overvaluing assets simply because they are owned.",
    "Fear-related emotion": "Words or tone reflecting emotional discomfort (e.g., fear, regret, anxiety)."
}

# Prompt constructor
def make_prompt(text):
    trait_desc = "\n".join([f"- {k}: {v}" for k, v in loss_aversion_traits.items()])
    return (
        "You are a behavioral finance expert.\n\n"
        "Your task is to analyze a social media post for **Loss Aversion** bias in personal financial decisions using the traits listed below.\n\n"
        "**Traits:**\n"
        f"{trait_desc}\n\n"
        "**Instructions:**\n"
        "- Identify which traits are present in the post.\n"
        "- Assign a numeric weight (from 0 to 1) to each trait you detect.\n"
        "- If at least one trait has weight > 0, classify Loss Aversion as 'Yes'. Otherwise 'No'.\n"
        "- Based on the dominant weights, also assign Intensity as High, Mid, or Low:\n"
        "   • High: weights ≥ 0.8\n"
        "   • Mid: weights between 0.4 and 0.7\n"
        "   • Low: weights below 0.4\n\n"
        "**Respond in this format:**\n"
        "Loss Aversion: <Yes/No>\n"
        "Intensity: <High/Mid/Low>\n"
        "Reasoning:\n"
        "- List the present traits, their weights, and justify each.\n"
        "- Conclude why you decided on Loss Aversion and Intensity.\n\n"
        f"Statement:\n{text}"
    )

# Robust response parser
def parse_response(text):
    try:
        loss_aversion = re.search(r"Loss Aversion:\s*(Yes|No)", text, re.IGNORECASE)
        intensity = re.search(r"Intensity:\s*(High|Mid|Low)", text, re.IGNORECASE)
        reasoning = re.search(r"Reasoning:\s*(.*?)(?:\n(?:Loss Aversion|Intensity|Statement|$)|$)", text, re.IGNORECASE | re.DOTALL)

        return {
            'Loss_Aversion': loss_aversion.group(1).capitalize() if loss_aversion else "",
            'Intensity': intensity.group(1).capitalize() if intensity else "",
            'Reasoning': reasoning.group(1).strip() if reasoning else ""
        }
    except Exception as e:
        return {
            'Loss_Aversion': "No",
            'Intensity': "Low",
            'Reasoning': f"⚠️ Parsing failed: {str(e)}"
        }

# Load data
df = pd.read_csv("author_334_posts.csv").iloc[1000:1025].copy()
if 'input_text' not in df.columns:
    raise ValueError("❌ 'input_text' column not found.")

df['Loss_Aversion'] = ""
df['Intensity'] = ""
df['Reasoning'] = ""

# API limit config
calls_per_minute = 30
max_daily_calls = 1000
call_count = 0
start_time = time.time()

# Main loop
for i, row in df.iterrows():
    if call_count >= max_daily_calls:
        print("🔒 Reached daily limit.")
        break

    if call_count > 0 and call_count % calls_per_minute == 0:
        elapsed = time.time() - start_time
        if elapsed < 60:
            print(f"⏳ Waiting {60 - elapsed:.2f}s to avoid rate limit...")
            time.sleep(60 - elapsed)
        start_time = time.time()

    prompt = make_prompt(row['input_text'])

    retries = 0
    while True:
        try:
            response = openai.ChatCompletion.create(
                model="meta-llama/llama-4-scout-17b-16e-instruct",
                messages=[
                    {"role": "system", "content": "You are a behavioral finance expert."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.1
            )
            call_count += 1
            text = response['choices'][0]['message']['content'].strip()
            print(f"\n📤 Row {i} Response:\n{text}\n")

            output = parse_response(text)
            for key in output:
                df.at[i, key] = output[key]
            break

        except Exception as e:
            if retries >= 3:
                df.at[i, 'Loss_Aversion'] = "No"
                df.at[i, 'Reasoning'] = f"Error: {str(e)}"
                break
            time.sleep(2 ** retries)
            retries += 1

    time.sleep(2)

# Save results
df.to_csv("experiment1_traits_only.csv", index=False)
print("✅ Saved to experiment1_traits_only.csv")



📤 Row 1000 Response:
Loss Aversion: Yes
Intensity: Mid

Reasoning:

The following traits of Loss Aversion are present in the post:

1. **Holding losses: Not selling underperforming assets to avoid the pain of realizing a loss.** (Weight: 0.6) 
The post implies that the author is hesitant to adjust their portfolio in response to changing market conditions, specifically the low dividend yields of Bond Funds. The author seems to be holding onto their current allocation, possibly to avoid realizing losses.

2. **Fear-related emotion: Words or tone reflecting emotional discomfort (e.g., fear, regret, anxiety).** (Weight: 0.7) 
The tone of the post reflects concern and anxiety about the potential impact of low dividend yields on returns. Phrases like "hurt returns," "scary now," and "problems start with basically no dividend" convey a sense of unease.

3. **Status quo bias: Preferring inaction to avoid the possibility of loss.** (Weight: 0.5) 
The author seems to be inclined to maintain the

In [ ]:
files.download('experiment1_traits_only.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import openai
import time
import re

# Define traits with definitions and keywords
loss_aversion_keywords = {
    "Holding losses": {
        "definition": "Not selling underperforming assets to avoid the pain of realizing a loss.",
        "keywords": ["still holding", "bagholding", "can't sell", "won't sell", "refuse to sell"]
    },
    "Temporal references": {
        "definition": "Mentioning past losses to justify current decisions.",
        "keywords": ["since I lost", "after the last crash"]
    },
    "Comparative framing": {
        "definition": "Highlighting what was lost versus what could have been gained.",
        "keywords": ["could have made", "should have sold", "missed the gains"]
    },
    "Status quo bias": {
        "definition": "Preferring inaction to avoid the possibility of loss.",
        "keywords": ["better to wait", "doing nothing is safer"]
    },
    "Endowment effect": {
        "definition": "Overvaluing assets simply because they are owned.",
        "keywords": ["feels wrong to sell", "personal attachment"]
    },
    "Fear-related emotion": {
        "definition": "Words or tone reflecting emotional discomfort.",
        "keywords": ["regret", "scared", "nervous", "anxious", "panicking", "worried"]
    }
}

# Prompt generator with keywords
def make_prompt(text):
    trait_definitions = ""
    for name, data in loss_aversion_keywords.items():
        keywords = ", ".join(f'"{kw}"' for kw in data['keywords'])
        trait_definitions += f"- {name}: {data['definition']}\n  Keywords: {keywords}\n"

    return (
        "You are a behavioral finance expert.\n\n"
        "Your task is to analyze the social media post below for **Loss Aversion** bias.\n"
        "Match any of the **keywords** under each trait only if they are used in the context of a personal investment decision.\n"
        "Assign a numeric weight (between 0 and 1) to each characteristic based on relevance and strength of match.\n\n"
        "**Behavioral Characteristics with Keywords:**\n"
        f"{trait_definitions}\n\n"
        "**Instructions:**\n"
        "- List which traits are present (with matched keywords).\n"
        "- Assign each present trait a numeric weight from 0 to 1 based on its relevance.\n"
        "- If any trait has a weight > 0, classify Loss Aversion: Yes. Else: No.\n"
        "- Also determine Intensity: High (≥ 0.8), Mid (0.4–0.7), Low (< 0.4).\n\n"
        "**Response format:**\n"
        "Loss Aversion: <Yes/No>\n"
        "Intensity: <High/Mid/Low>\n"
        "Reasoning:\n"
        "- List each trait, matched keywords, weights, and reasoning.\n\n"
        f"Statement:\n{text}"
    )

# Robust parser
def parse_response(text):
    try:
        loss_aversion = re.search(r"Loss Aversion:\s*(Yes|No)", text, re.IGNORECASE)
        intensity = re.search(r"Intensity:\s*(High|Mid|Low)", text, re.IGNORECASE)
        reasoning = re.search(r"Reasoning:\s*(.*?)(?:\n\s*(?:Loss Aversion|Intensity|Statement|$)|$)", text, re.IGNORECASE | re.DOTALL)

        return {
            'Loss_Aversion': loss_aversion.group(1).capitalize() if loss_aversion else "",
            'Intensity': intensity.group(1).capitalize() if intensity else "",
            'Reasoning': reasoning.group(1).strip() if reasoning else ""
        }
    except Exception as e:
        return {
            'Loss_Aversion': "No",
            'Intensity': "Low",
            'Reasoning': f"⚠️ Parsing failed: {str(e)}"
        }

# Load data
df = pd.read_csv("author_334_posts.csv").iloc[1000:1025].copy()
if 'input_text' not in df.columns:
    raise ValueError("❌ 'input_text' column not found.")

df['Loss_Aversion'] = ""
df['Intensity'] = ""
df['Reasoning'] = ""

# API limits
calls_per_minute = 30
max_daily_calls = 1000
call_count = 0
start_time = time.time()

# Main loop
for i, row in df.iterrows():
    if call_count >= max_daily_calls:
        print("🔒 Daily API limit reached.")
        break

    if call_count > 0 and call_count % calls_per_minute == 0:
        elapsed = time.time() - start_time
        if elapsed < 60:
            print(f"⏳ Waiting {60 - elapsed:.2f}s...")
            time.sleep(60 - elapsed)
        start_time = time.time()

    prompt = make_prompt(row['input_text'])

    retries = 0
    while True:
        try:
            response = openai.ChatCompletion.create(
                model="meta-llama/llama-4-scout-17b-16e-instruct",
                messages=[
                    {"role": "system", "content": "You are a behavioral finance expert."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.1
            )
            call_count += 1
            text = response['choices'][0]['message']['content'].strip()
            print(f"\n📤 Row {i} Response:\n{text}\n")

            parsed = parse_response(text)
            for key in parsed:
                df.at[i, key] = parsed[key]
            break

        except Exception as e:
            if retries > 3:
                df.at[i, 'Loss_Aversion'] = "No"
                df.at[i, 'Reasoning'] = f"Error: {e}"
                break
            time.sleep(2 ** retries)
            retries += 1

    time.sleep(2)

# Save output
df.to_csv("experiment2_traits_with_keywords.csv", index=False)
print("✅ Saved to experiment2_traits_with_keywords.csv")



📤 Row 1000 Response:
**Loss Aversion: Yes**
**Intensity: Mid**

Reasoning:

* **Holding losses: Not selling underperforming assets to avoid the pain of realizing a loss.** 
  - Matched keywords: None
  - Weight: 0
  - Reasoning: There is no mention of holding onto underperforming assets to avoid selling at a loss.

* **Temporal references: Mentioning past losses to justify current decisions.**
  - Matched keywords: None
  - Weight: 0
  - Reasoning: There is no mention of past losses justifying current decisions.

* **Comparative framing: Highlighting what was lost versus what could have been gained.**
  - Matched keywords: "hurt returns", "scary"
  - Weight: 0.6
  - Reasoning: The statement implies that the potential drop in dividend yields will "hurt returns," suggesting a concern about what could be lost. The use of "scary" also indicates a framing around potential negative outcomes.

* **Status quo bias: Preferring inaction to avoid the possibility of loss.**
  - Matched keywords: 

In [ ]:
files.download('experiment2_traits_with_keywords.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import openai
import time
import re

# Keywords and trait definitions
loss_aversion_keywords = {
    "Holding losses": {
        "definition": "Not selling underperforming assets to avoid the pain of realizing a loss.",
        "keywords": ["still holding", "bagholding", "can't sell", "won't sell", "refuse to sell"]
    },
    "Temporal references": {
        "definition": "Mentioning past losses to justify current decisions.",
        "keywords": ["since I lost", "after the last crash"]
    },
    "Comparative framing": {
        "definition": "Highlighting what was lost versus what could have been gained.",
        "keywords": ["could have made", "should have sold", "missed the gains"]
    },
    "Status quo bias": {
        "definition": "Preferring inaction to avoid the possibility of loss.",
        "keywords": ["better to wait", "doing nothing is safer"]
    },
    "Endowment effect": {
        "definition": "Overvaluing assets simply because they are owned.",
        "keywords": ["feels wrong to sell", "personal attachment"]
    },
    "Fear-related emotion": {
        "definition": "Emotional discomfort (fear, regret, anxiety).",
        "keywords": ["regret", "scared", "nervous", "anxious", "panicking", "worried"]
    }
}

def make_prompt(text):
    traits_text = ""
    for trait, info in loss_aversion_keywords.items():
        kws = ", ".join(f'"{kw}"' for kw in info["keywords"])
        traits_text += f"- {trait}: {info['definition']}\n  Keywords: {kws}\n"

    return (
        "You are a behavioral finance expert.\n\n"
        "Analyze the following social media post to detect **Loss Aversion** bias using both the listed keywords and **semantically similar phrases**.\n\n"
        "**Behavioral Characteristics and Keywords**:\n"
        f"{traits_text}\n\n"
        "**Instructions**:\n"
        "1. Identify each Characteristic that is present in the post (based on keywords or similar expressions).\n"
        "2. For every detected Characteristics, assign a **numeric weight (0–1)**.\n"
        "3. Format output as:\n"
        "   - <Trait>: <weight> → <matched phrase(s)> → <brief explanation>\n"
        "4. Set `Loss Aversion` to Yes if any trait weight > 0. Else No.\n"
        "5. Set `Intensity`:\n"
        "   - High , Mid , Low\n\n"
        "**Output Format**:\n"
        "Loss Aversion: <Yes/No>\n"
        "Intensity: <High/Mid/Low>\n"
        "Reasoning:\n"
        "- Trait 1: <weight> → <matched phrase> → <reason>\n"
        "- Trait 2: <weight> → ...\n"
        "- ...\n"
        "- Conclusion: <brief summary>\n\n"
        f"Statement:\n{text}"
    )

def parse_response(text):
    def extract(pattern):
        match = re.search(pattern, text, re.IGNORECASE)
        return match.group(1).strip() if match else ""

    return {
        'Loss_Aversion': extract(r"Loss Aversion:\s*(Yes|No)").capitalize(),
        'Intensity': extract(r"Intensity:\s*(High|Mid|Low)").capitalize(),
        'Reasoning': extract(r"Reasoning:\s*(.*)", re.DOTALL)
    }

# Load data
df = pd.read_csv("author_334_posts.csv").iloc[1000:1025].copy()
df[['Loss_Aversion', 'Intensity', 'Reasoning']] = ""

# API config
calls_per_minute = 30
max_daily_calls = 1000
call_count = 0
start_time = time.time()

for i, row in df.iterrows():
    if call_count >= max_daily_calls:
        break
    if call_count > 0 and call_count % calls_per_minute == 0:
        elapsed = time.time() - start_time
        if elapsed < 60:
            time.sleep(60 - elapsed)
        start_time = time.time()

    prompt = make_prompt(row['input_text'])
    retries = 0

    while True:
        try:
            response = openai.ChatCompletion.create(
                model="meta-llama/llama-4-scout-17b-16e-instruct",
                messages=[
                    {"role": "system", "content": "You are a behavioral finance expert."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.1
            )
            call_count += 1
            text = response['choices'][0]['message']['content'].strip()
            parsed = parse_response(text)
            for key in parsed:
                df.at[i, key] = parsed[key]
            break
        except Exception as e:
            if retries > 3:
                df.at[i, 'Loss_Aversion'] = "No"
                df.at[i, 'Reasoning'] = f"Error: {e}"
                break
            time.sleep(2 ** retries)
            retries += 1

    time.sleep(2)

df.to_csv("experiment3_semantic_traits.csv", index=False)
print("✅ Saved to experiment3_semantic_traits.csv")


✅ Saved to experiment3_semantic_traits.csv


In [ ]:
files.download('experiment3_semantic_traits.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import openai
import time
import re

# Keywords and trait definitions
loss_aversion_keywords = {
    "Holding losses": {
        "definition": "Not selling underperforming assets to avoid the pain of realizing a loss.",
        "keywords": ["still holding", "bagholding", "can't sell", "won't sell", "refuse to sell"]
    },
    "Temporal references": {
        "definition": "Mentioning past losses to justify current decisions.",
        "keywords": ["since I lost", "after the last crash"]
    },
    "Comparative framing": {
        "definition": "Highlighting what was lost versus what could have been gained.",
        "keywords": ["could have made", "should have sold", "missed the gains"]
    },
    "Status quo bias": {
        "definition": "Preferring inaction to avoid the possibility of loss.",
        "keywords": ["better to wait", "doing nothing is safer"]
    },
    "Endowment effect": {
        "definition": "Overvaluing assets simply because they are owned.",
        "keywords": ["feels wrong to sell", "personal attachment"]
    },
    "Fear-related emotion": {
        "definition": "Emotional discomfort (fear, regret, anxiety).",
        "keywords": ["regret", "scared", "nervous", "anxious", "panicking", "worried"]
    }
}

# Prompt creator with enhanced filter logic
def make_prompt(text):
    traits_text = ""
    for trait, info in loss_aversion_keywords.items():
        kws = ", ".join(f'"{kw}"' for kw in info["keywords"])
        traits_text += f"- {trait}: {info['definition']}\n  Keywords: {kws}\n"

    return (
        "You are a behavioral finance expert.\n\n"
        "Your task is to analyze the following social media post for **Loss Aversion** bias.\n\n"
        "----------------------------------\n"
        "**Step 1: Decision Relevance Filter**\n"
        "Read the post carefully and determine whether it reflects a **personal investment decision**.\n"
        "Only proceed to Step 2 if at least ONE of the following applies:\n"
        "1. The post includes a decision made too quickly or too late **with emotion or justification** (e.g., panic selling, holding despite regret).\n"
        "2. The post reacts to a **market event** and clearly includes a **personal investment action** or emotional reaction.\n"
        "3. The post includes **future-oriented thinking** with signs of **planning, worry, or conditional decisions**.\n"
        "If none of these apply, classify as informational:\n"
        "Loss Aversion: No\n"
        "Intensity: None\n"
        "Reasoning: Informational post only — no personal decision detected.\n\n"
        "----------------------------------\n"
        "**Step 2: Bias Trait Detection (only if post is decision-relevant)**\n"
        "**Behavioral Characteristics and Keywords**:\n"
        f"{traits_text}\n\n"
        "**Instructions**:\n"
        "1. Use both the keywords and semantically similar expressions.\n"
        "2. Identify each trait present and assign it a **numeric weight (0–1)**.\n"
        "3. If any trait has weight > 0, set `Loss Aversion: Yes`. Otherwise: No.\n"
        "4. Set `Intensity` as:\n"
        "   - High: multiple traits with weight > 0.7\n"
        "   - Mid: at least one trait between 0.4–0.7\n"
        "   - Low: only weak signals below 0.4\n"
        "   - None: for non-decision-based posts\n\n"
        "**Response Format**:\n"
        "Loss Aversion: <Yes/No>\n"
        "Intensity: <High/Mid/Low/None>\n"
        "Reasoning:\n"
        "- Trait 1: <weight> → <matched phrase> → <explanation>\n"
        "- Trait 2: <weight> → ...\n"
        "- Conclusion: <summarize why Loss Aversion is or isn’t present>\n\n"
        f"Statement:\n{text}"
    )

# Response parser
def parse_response(text):
    def extract(pattern, flags=0):
        match = re.search(pattern, text, flags)
        return match.group(1).strip() if match else ""

    return {
        'Loss_Aversion': extract(r"Loss Aversion:\s*(Yes|No)", re.IGNORECASE).capitalize(),
        'Intensity': extract(r"Intensity:\s*(High|Mid|Low|None)", re.IGNORECASE).capitalize(),
        'Reasoning': extract(r"Reasoning:\s*(.*)", re.DOTALL)
    }

# Load dataset
df = pd.read_csv("author_334_posts.csv").iloc[1000:1025].copy()
df[['Loss_Aversion', 'Intensity', 'Reasoning']] = ""

# API settings
calls_per_minute = 30
max_daily_calls = 1000
call_count = 0
start_time = time.time()

# Run model
for i, row in df.iterrows():
    if call_count >= max_daily_calls:
        break
    if call_count > 0 and call_count % calls_per_minute == 0:
        elapsed = time.time() - start_time
        if elapsed < 60:
            time.sleep(60 - elapsed)
        start_time = time.time()

    prompt = make_prompt(row['input_text'])
    retries = 0

    while True:
        try:
            response = openai.ChatCompletion.create(
                model="meta-llama/llama-4-scout-17b-16e-instruct",
                messages=[
                    {"role": "system", "content": "You are a behavioral finance expert."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.1
            )
            call_count += 1
            text = response['choices'][0]['message']['content'].strip()
            parsed = parse_response(text)
            for key in parsed:
                df.at[i, key] = parsed[key]
            print(f"\n✅ Row {i} Processed:\n{text}\n")
            break
        except Exception as e:
            if retries > 3:
                df.at[i, 'Loss_Aversion'] = "No"
                df.at[i, 'Reasoning'] = f"Error: {e}"
                break
            time.sleep(2 ** retries)
            retries += 1

    time.sleep(2)

# Save results
df.to_csv("experiment3_with_final_filters.csv", index=False)
print("✅ Saved to experiment3_with_final_filters.csv")



✅ Row 1000 Processed:
**Step 1: Decision Relevance Filter**
The post reflects a personal investment concern rather than a direct decision made. However, it implies future-oriented thinking with signs of planning and worry about the impact of changing interest rates on bond funds, which are a significant part of the author's portfolio. Therefore, it does relate to a personal investment situation.

**Decision Relevance:** Yes

**Step 2: Bias Trait Detection**

The post exhibits several characteristics that could indicate Loss Aversion:

- **Holding losses / Status quo bias:** The author seems concerned about the future performance of Intermediate Bond Funds but does not explicitly state an unwillingness to sell. However, there's an implicit preference for inaction or holding, as they question whether these funds "will do fine" without considering selling. 
  - Weight: 0.3 → "seem scary now" and "maybe they will do fine" → The author is hesitant but not directly stating an unwillingness 

In [ ]:
files.download('experiment3_with_final_filters.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>